In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from graphviz import Digraph


In [4]:
np.random.seed(1)
n = 1000

In [11]:
def draw_dag(edges, title):
    dot = Digraph(comment=title)
    for parent, child in edges:
        dot.edge(parent, child)
    display(dot)

First example - one confounder

In [12]:
draw_dag(
    edges=[
        ('Motivation', 'TrainingHours'),
        ('Motivation', 'Productivity')
    ],
    title='Productivity <- Motivation -> TrainingHours'
)

NameError: name 'Digraph' is not defined

In [8]:
motivation = np.random.normal(0, 1, n)
training_hours = 0.7 * motivation + np.random.normal(0, 1, n)
productivity = 2.0 * training_hours + 1.5 * motivation + np.random.normal(0, 1, n)

data1 = pd.DataFrame({
    'motivation': motivation,
    'training_hours': training_hours,
    'productivity': productivity
})

In [10]:
model1 = smf.ols('productivity ~ motivation + training_hours', data=data1).fit()
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:           productivity   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     6329.
Date:                Sat, 21 Jun 2025   Prob (F-statistic):               0.00
Time:                        17:44:51   Log-Likelihood:                -1410.3
No. Observations:                1000   AIC:                             2827.
Df Residuals:                     997   BIC:                             2841.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0510      0.031     -1.

Second example - two variable

Third example - collider

Fourth example - mediator